In [2]:
import cv2
import os
import sys
import math
from PIL.Image import fromarray
from IPython.display import Image, display
import matplotlib.pyplot as plt
from pathlib import Path
import copy
from ultralytics import YOLO

# YOLOの設定
model = YOLO('yolov8x.pt')

# 動画の読み込み
cap = cv2.VideoCapture("./video/dog.mp4")



In [3]:
# 中心領域を定義
def is_in_center_area(height):
    return height / 4 , 3 * height / 4

def imshow(img):
    ret, encoded = cv2.imencode(".jpg", img)
    display(Image(data=encoded.tobytes()))

In [ ]:
fps = cap.get(cv2.CAP_PROP_FPS)
fps

In [5]:
frame_list = []
ret = True
while ret:
    ret, frame = cap.read()
    if ret:
        frame_list.append(frame)

In [ ]:
# 総フレーム数を計算する
len(frame_list)

In [ ]:
# 700番目から800番目ののフレームを描画してみる
frame = frame_list[730]
imshow(frame)

In [ ]:
ret = model(frame)

In [ ]:
results = ret = model(frame)
target = {00000:"dogfood"}
dogfoods = [food for food in results[0].boxes if int(food.cls[0].item()) in target]

In [ ]:

# # クラスIDの確認
# class_names = model.names
# # クラスIDとクラス名の表示
# for idx, class_name in enumerate(class_names):
#     print(f"Class ID: {idx}, Class Name: {class_name}")

# # 結果を表示して確認
# for result in results:
#     boxes = result.boxes
#     print(f"Detected {len(boxes)} objects.")
#     for box in boxes:
#         cls = int(box.cls[0].item())
#         confidence = box.conf[0].item()
#         class_name = class_names[cls]
#         print(f"Class: {cls} ({class_name}), Confidence: {confidence}")

# # 対象物のフィルタリング
# target = {77: "teddy bears"}
# teddys = [box for box in results[0].boxes if int(box.cls[0].item()) in target]

# # 対象物がフィルタリングされているか確認
# print(f"Number of target objects: {len(teddys)}")
# for teddy in teddys:
#     cls = int(teddy.cls[0].item())
#     confidence = teddy.conf[0].item()
#     print(f"Filtered Class: {cls} ({target[cls]}), Confidence: {confidence}")

In [ ]:
teddys

In [69]:
# import cv2
# from IPython.display import Image, display
# import matplotlib.pyplot as plt
# from ultralytics import YOLO

# # YOLOの設定
# model = YOLO('yolov8x.pt')

# # 動画の読み込み
# cap = cv2.VideoCapture("./video/dog.mp4")

# # フレームをリストに追加
# frame_list = []
# ret = True
# while ret:
#     ret, frame = cap.read()
#     if ret:
#         frame_list.append(frame)

# # 70番目のフレームを描画
# def imshow(img):
#     ret, encoded = cv2.imencode(".jpg", img)
#     display(Image(data=encoded.tobytes()))

# frame = frame_list[700]
# imshow(frame)

# # モデルの推論
# results = model(frame)

# # 結果を表示して確認
# for result in results:
#     boxes = result.boxes
#     print(f"Detected {len(boxes)} objects.")
#     for box in boxes:
#         cls = int(box.cls[0].item())
#         confidence = box.conf[0].item()
#         print(f"Class: {cls}, Confidence: {confidence}")

# # 対象物のフィルタリング
# target = {2: "dog", 5: "bowl"}
# bowls = [box for box in results[0].boxes if int(box.cls[0].item()) in target]

# # 対象物がフィルタリングされているか確認
# print(f"Number of target objects: {len(bowls)}")
# for bowl in bowls:
#     cls = int(bowl.cls[0].item())
#     confidence = bowl.conf[0].item()
#     print(f"Filtered Class: {cls}, Confidence: {confidence}")



In [ ]:
# 検出結果の表示用にコピー
f_view = copy.copy(frame)
for food in dogfoods:
    # bboxは車の左上の座標と右下の座標を持つ
    bbox = {}
    bbox["x1"] = food.xyxy[0][0]
    bbox["y1"] = food.xyxy[0][1]
    bbox["x2"] = food.xyxy[0][2]
    bbox["y2"] = food.xyxy[0][3]
    cv2.rectangle(
        f_view,
        (int(bbox["x1"]), int(bbox["y1"])),
        (int(bbox["x2"]), int(bbox["y2"])),
        color=(0, 255, 0),
        thickness=2,
    )

imshow(f_view)

In [ ]:
# カウントする範囲（Y軸のみ）を設定
WINDOW_START=150
WINDOW_END  =450

In [ ]:
# ドッグフードクラスを作る
class dogfood:
    def __init__(self, bbox, f):
        self.bbox = bbox
        x1=round(bbox["x1"])
        x2=round(bbox["x2"])
        y1=round(bbox["y1"])
        y2=round(bbox["y2"])
        self.img  = copy.copy(f[y1:y2, x1:x2])
        return
    
    def show(self):
        imshow(self.img)
    
    def compare_by_hist(self, another_car):
        # histgramで比較
        self_hist = cv2.calcHist([self.img], [0], None, [256], [0, 256])
        another_hist = cv2.calcHist([another_car.img], [0], None, [256], [0, 256])
        return 1-cv2.compareHist(self_hist, another_hist, 0)
    
    def compare_by_pos(self, another_car):
        # 位置で比較
        x0 = round((self.bbox["x1"]+self.bbox["x2"])/2)
        y0 = round((self.bbox["y1"]+self.bbox["y2"])/2)
        x1 = round((another_car.bbox["x1"]+another_car.bbox["x2"])/2)
        y1 = round((another_car.bbox["y1"]+another_car.bbox["y2"])/2)
        dx = x1-x0
        dy = y1-y0
        return math.sqrt(dx*dx + dy*dy)
    
    def compare(self, another_car):
        # 2つを組み合わせて比較
        return abs(self.compare_by_hist(another_car) * self.compare_by_pos(another_car))

In [ ]:
# カウントする範囲（Y軸のみ）を設定
WINDOW_START=150
WINDOW_END  =450
car_set = []
INSET_THRESHOLD = 8
food_sum = 0
pictures = []
for i in range(len(frame_list)):
    if i % 3 >= 1:
        continue
    f = frame_list[i]
    results = ret = model(f)
    #ドッグフードを学習させたyoloを用いる
    target = {00000:"dogfood"}
    dogfoods = [food for food in results[0].boxes if int(food.cls[0].item()) in target]

    # 検出結果の表示用にコピー
    f_view = copy.copy(f)
    # カウントする範囲を描画
    cv2.rectangle(f_view,(0,WINDOW_START), (320, WINDOW_END), color=(0, 0, 255),thickness=2)
    pre_food_set = food_set
    food_set = []
    
    #検知範囲の限定
    for food in dogfoods:
        bbox = {}
        bbox["x1"] = food.xyxy[0][0].item()
        bbox["y1"] = food.xyxy[0][1].item()
        bbox["x2"] = food.xyxy[0][2].item()
        bbox["y2"] = food.xyxy[0][3].item()
        ypos = round((bbox["y1"]+bbox["y2"])/2)
        if ypos >= WINDOW_START and ypos <= WINDOW_END:
            # 同じフレームで2重カウントしてなかったら追加
            inSet = False
            food = dogfoods(bbox, f)
            for c in food_set:
                if food.compare(c) < INSET_THRESHOLD:
                    inSet = True
            if not inSet:
                food_set.append(food)
                # 描画
                cv2.rectangle(
                    f_view,
                    (round(bbox["x1"]), round(bbox["y1"])),
                    (round(bbox["x2"]), round(bbox["y2"])),
                    color=(0, 255, 0),
                    thickness=2,
                )
                # 前フレームにもいなかったら新しい車としてcar_sumを1増やす
                inSet = False
                for c in pre_food_set:
                    if food.compare(c) < INSET_THRESHOLD:
                        inSet = True
                if not inSet:
                    food_sum += 1
                    #通過した車が何台目かカウント
                    print(food_sum)
    imshow(f_view)
    a = cv2.cvtColor(copy.copy(f_view), cv2.COLOR_BGR2RGB)
    pictures.append(fromarray(a))